### IBM Applied Data Science Capstone
### Segmenting and Clustering Neighborhoods in Toronto
### Week 3 Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto
The purpose of this assignment is to explore and cluster the neighborhoods in Toronto.

## Part I: Getting the data into a dataframe

### Import libraries and general preparation:

In [231]:
#Import the necessary libraries:
#For part I:
from bs4 import BeautifulSoup # this module helps in web scrapping.
import requests  # this module helps us to download a web page
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#Starting in part II:
#!pip install geocoder
import geocoder
import time

#Starting in part III:
from geopy.geocoders import Nominatim
import folium
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

### Get the relevant data from Wikipedia into a table:

In [2]:
#Note the correct web address for the Toronto data of interest:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#Get the webpage's data in a text format:
data  = requests.get(url).text
#Creat a beautiful soup objectL
soup = BeautifulSoup(data,"html5lib")
#Get the relevant table data into a dataframe:
tables = soup.find_all('table')
#len(tables)
#Note the first table on the wikipedia page is the one of interest:
table=tables[0]
#table

### Prepare a properly formatted dataframe and populate it with the Wikipedia data:

In [35]:
#Prepare the destination table:
table_contents=[]
#Sort through the table from the prior step row by row:
for row in table.findAll('td'):
    cell = {}
    #If row of cells does not have an assigned borough, ignore them:
    if row.span.text=='Not assigned':
        pass
    #Else grab the appropriate text and store in dictionary 'cell':
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        #If a cell has a borough but neighborhood is 'Not assigned', then neighborhood=borough:
        if cell['Neighborhood']=='Not assigned' :
            cell['Neighborhood']=cell['Borough']
        #Now append the completed dictionary to destination table_contents:
        table_contents.append(cell)
#Convert into a pandas dataframe:
df=pd.DataFrame(table_contents)
#Do some final cleaning up of inconsistent names:
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto Business,Enclave of M4L
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [4]:
df.shape

(103, 3)

# Part II: Get geocoordinates for each neighborhood

### First define a function that uses geocoder to search for coordinates by postal code:
Note if the coordinates aren't found within a specified time, it returns none.

In [32]:
#Turn the course's recommended use of geocoder into a function and add a time constraint for when it doesn't return a set of coordinates:
def use_geocoder(postal_code_in):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates or a specific amount of times has gone by:
    t_end = time.time() + 0.25
    while (lat_lng_coords is None) and (time.time() < t_end) :
            #Note, use .arcgis instead of .google:
            g = geocoder.arcgis('{}, Toronto, Ontario'.format(postal_code_in))
            lat_lng_coords = g.latlng
    #If geocoder was successful, return the coordinates:
    if lat_lng_coords != None :
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    #Else fill in the coordinates with None:
    else :
        latitude = None
        longitude = None
    #Return the result which is either coordinates or (None, None):
    return (postal_code_in, latitude, longitude)

### Then use the function to retrieve geocoordinates for each neighborhood if available:

In [147]:
#Go through the list of Toronto postal codes and retrieve geocoordinates using the function defined above:
arcgis_geocoords=[]
for index, row in df.iterrows():
    #print("Index:", index)
    #print("PostalCode:", row["PostalCode"])
    #print(use_geocoder(row["PostalCode"]))
    arcgis_geocoords.append(use_geocoder(row["PostalCode"]))

#Then turn the result into a pandas dataframe:
main_coords=pd.DataFrame(arcgis_geocoords)
main_coords.columns=['PostalCode', 'Latitude', 'Longitude']
main_coords.head()

,PostalCode,Latitude,Longitude
0,M3A,43.75245,-79.32991
1,M4A,43.73057,-79.31306
2,M5A,43.65512,-79.36264
3,M6A,43.72327,-79.45042
4,M7A,43.66253,-79.39188


### In case no coordinates are returned, a backup dictionary is made from the .csv provided:

In [142]:
#Import the backup .csv as a pandas dataframe:
backup_coords = pd.read_csv('https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/labs_v1/Geospatial_Coordinates.csv')
#backup_coords.head()
#Make the postal codes, latitudes, and longitudes into a dictionary to reference easier in the next step:
backup_dict={}
for index, row in backup_coords.iterrows():
    backup_dict[row["Postal Code"]]=[row["Latitude"], row["Longitude"]]
#backup_dict

### Here, go through the postal codes from part I and assign coordinates to each neighborhood
If coordinates were returned from geocoder, it uses them. Else it uses the coordinates from the .csv backup file.

In [156]:
#Left merge the coodinates data from the geocoder search with the postal code, borough, and neighborhood data:
combined_df = pd.merge(df, main_coords, left_on = 'PostalCode', right_on = 'PostalCode', how='left')
#Now, using the dictionary form of the coordinates in .csv form provided, fill in any blanks:
for index, row in combined_df.iterrows():
    if row["Latitude"]=='' or row["Longitude"]=='':
        row["Latitude"]= backup_dict[row["PostalCode"]][0]
        row["Longitude"]= backup_dict[row["PostalCode"]][1]
        print("missing coordinates, using backup file!")
combined_df

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188
5,M9A,Etobicoke,Islington Avenue,43.66263,-79.52831
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
7,M3B,North York,Don Mills North,43.74923,-79.36186
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804


# Part III: Explore and cluster the neighborhoods in Toronto

### First some summary data
The number of neighborhoods, boroughs, and Toronto's coordinates:

In [152]:
#Use the data developed above, plus the mapping framework from the New York exercise:
neighborhoods=combined_df.copy()
#Summary of the data:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)
#Get some starting coordinates for the mapping exercise:
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(address, latitude, longitude))

The dataframe has 15 boroughs and 103 neighborhoods.
The geograpical coordinate of Toronto, Canada are 43.6534817, -79.3839347.


### Visualizing the entire (103 row) dataset:

In [155]:
# create map of New York using latitude and longitude values
map_Toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto)  
    
map_Toronto

### Filtering down to the 'Toronto' data:
These are the Boroughs that contain the word "Toronto"

In [402]:
#Toronto_data=neighborhoods.copy()
Toronto_data = neighborhoods[neighborhoods['Borough'].str.contains('Toronto')].reset_index(drop=True)
Toronto_data

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M4E,East Toronto,The Beaches,43.67709,-79.29547
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306
5,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
6,M6G,Downtown Toronto,Christie,43.66869,-79.42071
7,M5H,Downtown Toronto,"Richmond, Adelaide, King",43.64970,-79.38258
8,M6H,West Toronto,"Dufferin, Dovercourt Village",43.66505,-79.43891
9,M4J,East York/East Toronto,The Danforth East,43.68811,-79.33418


### Map the Toronto proper data:

In [403]:
# create map of Toronto proper using latitude and longitude values
map_Toronto_proper = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(Toronto_data['Latitude'], Toronto_data['Longitude'], Toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_Toronto_proper)  
map_Toronto_proper

### Define Foursquare Credentials and Version:

In [404]:
#This square should be hidden when sharing:
CLIENT_ID = HIDDEN # your Foursquare ID
CLIENT_SECRET = HIDDEN # your Foursquare Secret

In [405]:
#Establish the search parameters:
VERSION = '20180605'
LIMIT=100
radius=500

### Define a function that searches Foursquare for each neighborhood:

In [406]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)     
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)    
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

### Run the function for each neighborhood: 

In [407]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighborhood'],
                                   latitudes=Toronto_data['Latitude'],
                                   longitudes=Toronto_data['Longitude']
                                  )

Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Rosedale
Enclave of M5E
St. James Town, Cabbagetown
First Canadi

### Get summary data on the resulting dataframe:

In [408]:
#Check the output size and format:
print(Toronto_venues.shape)
Toronto_venues.head()

(1680, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
1,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
2,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,"Regent Park, Harbourfront",43.65512,-79.36264,Body Blitz Spa East,43.654735,-79.359874,Spa


In [409]:
#Get the number of categories and neighborhood observation counts:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))
Toronto_venues.groupby('Neighborhood')['Neighborhood'].count()

There are 224 uniques categories.


Neighborhood
Berczy Park                                                                                                    64
Brockton, Parkdale Village, Exhibition Place                                                                   83
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport     76
Central Bay Street                                                                                             59
Christie                                                                                                       11
Church and Wellesley                                                                                           81
Commerce Court, Victoria Hotel                                                                                100
Davisville                                                                                                     26
Davisville North                                                           

### Prepare to analyze the data using dummy variable encoding:

In [410]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")
# # add neighborhood column back to dataframe
# Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 
# # move neighborhood column to the first column
# fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
# Toronto_onehot = Toronto_onehot[fixed_columns]

#Add neighborhood names back to the dataframe:
Toronto_onehot = Toronto_onehot.drop(['Neighborhood'], axis=1)
Toronto_onehot.insert(0, 'Neighborhood', Toronto_venues['Neighborhood'])

print(Toronto_onehot.shape)
Toronto_onehot.head()

(1680, 224)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [411]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
print(Toronto_grouped.shape)
Toronto_grouped.head()

(39, 224)


,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bike Trail,Bistro,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Line,Business Service,Butcher,Café,Camera Store,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Costume Shop,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts School,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Park,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recreation Center,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Club,Stadium,Steakhouse,Strip Club,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,Berczy Park,0.000000,0.0,0.0,0.0,0.015625,0.0,0.015625,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.015625,0.046875,0.000000,0.000000,0.0,0.031250,0.0,0.0,0.0,0.015625,0.0,0.000000,0.0,0.0,0.031250,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.015625,0.0,0.000000,0.000000,0.03125,0.0,0.000000,0.0,0.0,0.000000,0.015625,0.062500,0.093750,0.0,0.000000,0.0,0.0,0.015625,0.000000,0.015625,0.0,0.000000,0.0,0.015625,0.0,0.0,0.000000,0.0,0.015625,0.0,0.015625,0.000000,0.0,0.0,0.0,0.0,0.015625,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.03125,0.000000,0.0,0.015625,0.0,0.0,0.000000,0.0,0.0,0.0,0.015625,0.015625,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.015625,0.015625,0.015625,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.015625,0.000000,0.0,0.000000,0.015625,

### Print each neighborhood's top 5 venue types:

In [412]:
num_top_venues = 5
for hood in Toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.06
2              Bakery  0.05
3  Seafood Restaurant  0.05
4            Beer Bar  0.03


----Brockton, Parkdale Village, Exhibition Place----
            venue  freq
0             Bar  0.07
1            Café  0.06
2     Coffee Shop  0.05
3      Restaurant  0.05
4  Sandwich Place  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
                venue  freq
0  Italian Restaurant  0.08
1         Coffee Shop  0.07
2                Café  0.05
3                Park  0.04
4   French Restaurant  0.04


----Central Bay Street----
            venue  freq
0     Coffee Shop  0.12
1  Clothing Store  0.07
2     Pizza Place  0.05
3           Plaza  0.03
4  Cosmetics Shop  0.03


----Christie----
           venue  freq
0           Café  0.27
1  Grocery Store  0.18
2     Playground  0.09
3    Coffee Shop  0.09
4    Candy Store  0.0

In [413]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [414]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']
for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)
neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Pharmacy,Breakfast Spot,Cheese Shop,Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Bar,Café,Coffee Shop,Restaurant,Gift Shop,Sandwich Place,Japanese Restaurant,Supermarket,Furniture / Home Store,French Restaurant
2,"CN Tower, King and Spadina, Railway Lands, Har...",Italian Restaurant,Coffee Shop,Café,Bar,Park,Sandwich Place,French Restaurant,Bakery,Lounge,Gym / Fitness Center
3,Central Bay Street,Coffee Shop,Clothing Store,Pizza Place,Cosmetics Shop,Middle Eastern Restaurant,Sandwich Place,Bubble Tea Shop,Plaza,Restaurant,Juice Bar
4,Christie,Café,Grocery Store,Coffee Shop,Athletics & Sports,Playground,Italian Restaurant,Candy Store,Baby Store,Eastern European Restaurant,Electronics Store


### Clustering neighborhoods using k-means clustering:

In [415]:
# set number of clusters
kclusters = 5
Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:len(Toronto_grouped)]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 4, 1, 1, 0, 1, 4,
       0, 1, 1, 1, 0, 3, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1])

### create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [416]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Toronto_merged = Toronto_data.copy()
# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood:
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Toronto_merged.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,1,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804,1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Italian Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Theater,Diner
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587,1,Coffee Shop,Hotel,Cosmetics Shop,Gastropub,Cocktail Bar,Clothing Store,Café,Italian Restaurant,Seafood Restaurant,Park
3,M4E,East Toronto,The Beaches,43.67709,-79.29547,1,Health Food Store,Pub,Trail,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Pharmacy,Breakfast Spot,Cheese Shop,Restaurant


### Visualize the clustering result:

In [417]:
# create map:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)
# set color scheme for the clusters:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
# add markers to the map:
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)       
map_clusters

In [418]:
# for i in range(kclusters):
#     print("---- Cluster number {} of {} ----".format(i+1, kclusters))
#     print(i)
#     print(Toronto_merged.loc[Toronto_merged['Cluster Labels'] == i, Toronto_merged.columns[[1] + [2] + list(range(5, Toronto_merged.shape[1]))]])

### First cluster's data:

In [419]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[1] + [2] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
9,East York/East Toronto,The Danforth East,0,Park,Recreation Center,Intersection,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
18,Central Toronto,Lawrence Park,0,Bus Line,Park,Swim School,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
23,Central Toronto,North Toronto West,0,Playground,Gym Pool,Park,Distribution Center,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant,Escape Room
33,Downtown Toronto,Rosedale,0,Park,Playground,Bike Trail,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space


### Second cluster:

In [420]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[1] + [2] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,"Regent Park, Harbourfront",1,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Health Food Store,Italian Restaurant,Food Truck,Event Space,Electronics Store,Distribution Center
1,Downtown Toronto,"Garden District, Ryerson",1,Coffee Shop,Clothing Store,Café,Cosmetics Shop,Italian Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Hotel,Theater,Diner
2,Downtown Toronto,St. James Town,1,Coffee Shop,Hotel,Cosmetics Shop,Gastropub,Cocktail Bar,Clothing Store,Café,Italian Restaurant,Seafood Restaurant,Park
3,East Toronto,The Beaches,1,Health Food Store,Pub,Trail,Dog Run,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant
4,Downtown Toronto,Berczy Park,1,Coffee Shop,Cocktail Bar,Seafood Restaurant,Bakery,Beer Bar,Farmers Market,Pharmacy,Breakfast Spot,Cheese Shop,Restaurant
5,Downtown Toronto,Central Bay Street,1,Coffee Shop,Clothing Store,Pizza Place,Cosmetics Shop,Middle Eastern Restaurant,Sandwich Place,Bubble Tea Shop,Plaza,Restaurant,Juice Bar
6,Downtown Toronto,Christie,1,Café,Grocery Store,Coffee Shop,Athletics & Sports,Playground,Italian Restaurant,Candy Store,Baby Store,Eastern European Restaurant,Electronics Store
7,Downtown Toronto,"Richmond, Adelaide, King",1,Coffee Shop,Hotel,Café,Restaurant,Gym,Asian Restaurant,Japanese Restaurant,American Restaurant,Steakhouse,Salad Place
8,West Toronto,"Dufferin, Dovercourt Village",1,Park,Athletics & Sports,Bus Line,Middle Eastern Restaurant,Liquor Store,Furniture / Home Store,Café,Bar,Bank,Bakery
10,Downtown Toronto,"Harbourfront East, Union Station, Toronto Islands",1,Coffee Shop,Hotel,Japanese Restaurant,Plaza,Aquarium,Park,Deli / Bodega,Boat or Ferry,Music Venue,IT Services


### Third cluster:

In [421]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[1] + [2] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Central Toronto,Forest Hill North & West,2,Park,Yoga Studio,Donut Shop,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


### Fourth cluster:

In [422]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[1] + [2] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Central Toronto,Roselawn,3,Home Service,Fast Food Restaurant,Yoga Studio,Donut Shop,Fish & Chips Shop,Farmers Market,Farm,Falafel Restaurant,Event Space,Ethiopian Restaurant


### Fifth (last) cluster:

In [423]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[1] + [2] + list(range(5, Toronto_merged.shape[1]))]]

,Borough,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,West Toronto,"High Park, The Junction South",4,Park,Residential Building (Apartment / Condo),Convenience Store,Yoga Studio,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant,Event Space
29,Central Toronto,"Moore Park, Summerhill East",4,Tennis Court,Gym,Park,Convenience Store,Creperie,Donut Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
